# Subword Tokenizer

| **Tokenizer 방식** | **토큰 단위**                      | **vocab size** | **미등록 단어에 대한 가정**                                                                                  |
|---------------------|------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------|
| **사전 기반**       | 알려진 단어/형태소의 결합           | unlimited       | - 알려진 단어/형태소의 결합이라고 가정<br>- 필요한 형태소 분석 가능<br>- 사전에 등록되지 않은 단어는 UNK 처리 |
| **sub-word**        | 알려진 글자 및 sub-word            | fixed           | - 알려진 sub-words로 분해<br>- 예: appear → app + ear<br>- 자주 등장하는 단어를 제대로 인식 가능<br>- UNK의 개수 최소화 |

In [5]:
# 네이버 영화 리뷰 데이터

import urllib.request
import os

def get_file(filename, origin):
    cache_dir = os.path.expanduser('~/.torch/datasets')
    os.makedirs(cache_dir, exist_ok=True)           # exist_ok=True: 이미 폴더가 있어도 충돌없이 넘어갈 수 있게 설정
    filepath = os.path.join(cache_dir, filename)    # 원래 지정해준 cache_dir 경로에 filename까지 합쳐서 filepath로 저장됨

    if not os.path.exists(filepath):
        print(f'Downloading data from {origin}')
        urllib.request.urlretrieve(origin, filepath)
    
    return filepath

ratings_train_path = get_file('ratings_train.txt', "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
ratings_test_path = get_file('ratings_test.txt', "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")

ratings_train_path, ratings_test_path

('C:\\Users\\Playdata/.torch/datasets\\ratings_train.txt',
 'C:\\Users\\Playdata/.torch/datasets\\ratings_test.txt')

In [7]:
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep='\t', encoding='utf-8')
ratings_test_df = pd.read_csv(ratings_test_path, sep='\t', encoding='utf-8')

display(ratings_train_df)
display(ratings_test_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [ ]:
# 결측치 제거

ratings_train_df = ratings_train_df.dropna(how='any')   # how='any': 결측치가 하나라도 있으면 row를 삭제
ratings_test_df = ratings_test_df.dropna(how='any')

ratings_train_df.shape, ratings_test_df.shape

((149995, 3), (49997, 3))

In [10]:
# txt 파일 생성 - 학습 데이터
with open('naver_review.txt', 'w', encoding='utf-8') as f:
    for doc in ratings_train_df['document'].values:
        f.write(doc+'\n')       # 줄바꿈 추가해서 한 줄씩 저장되도록 코드 작성

### SentencePieceTokenizer

In [ ]:
# !pip install sentencepiece
# subword로 토큰화할 때 사용하는 라이브러리

   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   --------------------------------------- 992.0/992.0 kB 11.8 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm

input = 'naver_review.txt'
vocab_size = 10000              # subword의 vocabulary의 사이즈를 설정 (최대 만개까지 지정 가능함)
model_prefix = 'naver_review'   # 모델과 어휘 사전 파일의 접두어 설정
cmd = f'--input={input} --model_prefix={model_prefix} --vocab_size={vocab_size}'    # sentencepiece에 넘길 문자열 만드는 부분

spm.SentencePieceTrainer.Train(cmd) # cmd에 정의된대로 naver_review.txt파일을 읽고, 서브워드 모델을 학습한 뒤 결과물을 naver_review.model과 naver_review.vocab이라는 두 파일에 저장

In [14]:
sp = spm.SentencePieceProcessor()
sp.Load(f'{model_prefix}.model')    # 학습된 모델 파일을 로드해옴

for doc in ratings_train_df['document'].values[:3]:
    print(doc)
    print(sp.encode_as_pieces(doc))
    print(sp.encode_as_ids(doc))
    print()

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
[62, 877, 5, 31, 2019, 68, 1710]

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터', '보고', '▁초딩', '영화', '줄', '....', '오', '버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1634, 8, 4908, 159, 1460, 33, 264, 60, 173, 548, 410, 1224, 7396, 754, 440]

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보', '는것을', '추천', '한다']
[23, 369, 9781, 429, 3780, 143, 6266, 1945, 314]



In [ ]:
text = ratings_test_df['document'][100]
tokens = sp.encode_as_pieces(text)  # 인코드되면 서브워드 토크나이징된 결과를 볼 수 있음음
id_tokens = sp.encode_as_ids(text)
print(text)
print(tokens)
print(id_tokens)
print()

print(''.join(tokens).replace('_', ' ').strip())        # _는 유니코드 # 단어의 경계를 유니코드로 표시해줌
print(sp.decode_pieces(tokens))     # 서브 토큰으로 나눠진 것을 원본 텍스트로 복원 (decode)
print(sp.decode_ids(id_tokens))

걸작은 몇안되고 졸작들만 넘쳐난다.
['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
[1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]

▁걸작은▁몇안되고▁졸작들만▁넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.


In [19]:
print(u'\u2581')

▁


### BertwordPieceTokenizer

In [ ]:
# !pip install tokenizers

In [22]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)        # 한국어는 대소문자가 없고, 액센트가 없어서 옵션을 false로 둠
vocab_size = 10000      # 단어 10000개 저장

tokenizer.train(
    files = ['naver_review.txt'],
    vocab_size=vocab_size,
    min_frequency=5,        # 최소 5번 이상 등장한 단어만 subword vocab에 포함
    show_progress=True      # 학습 진행 상황 보여주기
)

In [23]:
encoded = tokenizer.encode('아버지가 방에 들어가신다.')
print(encoded.tokens)
print(encoded.ids)

['아버지가', '방', '##에', '들어가', '##신', '##다', '.']
[8316, 483, 1054, 6766, 1288, 1011, 16]


In [24]:
tokenizer.decode(encoded.ids)

'아버지가 방에 들어가신다.'